In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-29 11:46:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-06-29 11:46:21 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [16]:
vocab_size = len(chars)
chars = sorted(list(set(text)))
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
n = int(0.9*len(data))
train_data = data[:n] # On sépare 90% en train data
val_data = data[n:] # 10% en eval data

torch.Size([1115394]) torch.int64


In [10]:
block_size = 8 # context windows
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{context} - {target}")

tensor([18]) - 47
tensor([18, 47]) - 56
tensor([18, 47, 56]) - 57
tensor([18, 47, 56, 57]) - 58
tensor([18, 47, 56, 57, 58]) - 1
tensor([18, 47, 56, 57, 58,  1]) - 15
tensor([18, 47, 56, 57, 58,  1, 15]) - 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) - 58


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # context window

def get_batch(split):
    # generate batch
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"{context.tolist()} - {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
[24] - 43
[24, 43] - 58
[24, 43, 58] - 5
[24, 43, 58, 5] - 57
[24, 43, 58, 5, 57] - 1
[24, 43, 58, 5, 57, 1] - 46
[24, 43, 58, 5, 57, 1, 46] - 43
[24, 43, 58, 5, 57, 1, 46, 43] - 39
[44] - 53
[44, 53] - 56
[44, 53, 56] - 1
[44, 53, 56, 1] - 58
[44, 53, 56, 1, 58] - 46
[44, 53, 56, 1, 58, 46] - 39
[44, 53, 56, 1, 58, 46, 39] - 58
[44, 53, 56, 1, 58, 46, 39, 58] - 1
[52] - 58
[52, 58] - 1
[52, 58, 1] - 58
[52, 58, 1, 58] - 46
[52, 58, 1, 58, 46] - 39
[52, 58, 1, 58, 46, 39] - 58
[52, 58, 1, 58, 46, 39, 58] - 1
[52, 58, 1, 58, 46, 39, 58, 1] - 46
[25] - 17
[25, 17] - 27
[25, 17, 27] - 10
[25, 17, 2

In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module) :
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B=batch_size, T=context_window, C=vocab_size)
        if targets is None :
            loss = None
        else : 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [24]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # Optimizer with learning rate

In [31]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4606239795684814


In [32]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


EThand t, aghath,
MI: Fol'storevend os tho geat; tisheer lid al'st bal l are, tour bu olll pt my.
FR
